In [4]:
#!/usr/bin/env python
# coding: utf-8

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# List of years
year_list = [i for i in range(1980, 2024, 1)]

# Layout of the dashboard
app.layout = html.Div([
    html.H1(
        "XYZ Automobiles - Interactive Dashboard",
        style={
            'textAlign': 'center',
            'color': '#503D36',
            'fontSize': '24px',
            'fontFamily': 'Arial, sans-serif',
            'marginBottom': '20px'
        }
    ),

    html.Div([
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='stat-dropdown',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly'},
                {'label': 'Recession Period Statistics', 'value': 'Recession'}
            ],
            value='Yearly',
            placeholder='Select a statistic type'
        )
    ], style={'width': '40%', 'margin': 'auto', 'marginTop': '20px'}),

    html.Div([
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': year, 'value': year} for year in year_list],
            value=year_list[0],
            placeholder='Select a year'
        )
    ], style={'width': '40%', 'margin': 'auto', 'marginTop': '20px'}),

    html.Div(id='output-container', className='output-container', style={'marginTop': '30px'})
])

# Callback to enable/disable year dropdown based on statistic type
@app.callback(
    Output('select-year', 'disabled'),
    Input('stat-dropdown', 'value')
)
def update_input_container(selected_statistics):
    return False if selected_statistics == 'Yearly' else True

# Callback to update graphs based on dropdown selections
@app.callback(
    Output('output-container', 'children'),
    [Input('stat-dropdown', 'value'), Input('select-year', 'value')]
)
def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession':
        recession_data = data[data['Recession'] == 1]

        # Plot 1: Average sales over recession years
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(figure=px.line(yearly_rec, x='Year', y='Automobile_Sales',
                                            title='Average Automobile Sales over Recession Years'))

        # Plot 2: Avg vehicles sold by vehicle type
        avg_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(figure=px.bar(avg_sales, x='Vehicle_Type', y='Automobile_Sales',
                                           title='Avg Vehicles Sold by Type During Recession'))

        # Plot 3: Pie chart for ad expenditure share by type
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(figure=px.pie(exp_rec, names='Vehicle_Type', values='Advertising_Expenditure',
                                           title='Ad Expenditure Share During Recession'))

        # Plot 4: Unemployment effect
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                                           title='Unemployment Rate vs Sales by Vehicle Type'))

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)], style={'display': 'flex'})
        ]

    elif selected_statistics == 'Yearly' and input_year:
        yearly_data = data[data['Year'] == input_year]

        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales',
                                            title='Yearly Average Automobile Sales'))

        mas = data.groupby('Month')['Automobile_Sales'].mean().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas, x='Month', y='Automobile_Sales',
                                            title='Monthly Automobile Sales'))

        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales',
                                           title=f'Average Vehicles Sold in {input_year}'))

        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data, names='Vehicle_Type', values='Advertising_Expenditure',
                                           title='Ad Expenditure Share'))

        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)], style={'display': 'flex'})
        ]

    else:
        return html.Div("No data available.")

if __name__ == '__main__':
    app.run(debug=True)

